## The code will run with following GPU configurations 

In [16]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Jun  1 14:17:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    33W / 250W |  13223MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Updated libraries used in the code 

In [1]:
# !pip install --upgrade sentencepiece
# !pip install --upgrade datasets
# !pip install --upgrade transformers
# !pip install --upgrade rouge-score

In [2]:
import pandas as pd
import numpy as np
from datasets import *
from rouge_score import rouge_scorer

from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_metric


### Loading datasets
Load the Test.csv and Train.csv after downloading from Zindi after initializing the kernel.
<br>
Load the "french_fongbe_train_xlm_roberta.csv" and "french_fongbe_train_labse.csv" which was attained after running the code from step 3 of the documentation. These are going to be the additional data used to augment the given competition train data

In [3]:
test_data  = pd.read_csv('/content/Test.csv')
test_df    = test_data[test_data['Target_Language']=='Fon']

####### Loading existing train data and the extracted data from JW300
train_data  = pd.read_csv('/content/Train.csv')
train_data  = train_data[train_data['Target_Language']=='Fon']
train_data  = train_data.rename({'Target':'Fongbe'},axis=1)

extra_train = pd.read_csv('/content/french_fongbe_train_xlm_roberta.csv')

train_df    = train_data.append(extra_train).reset_index(drop=True)

#### train_df is the final train data to be used for training 

#### One of the datasets extracted form jw300 that will act as validation dataset
valid_df    = pd.read_csv('/content/french_fongbe_train_labse.csv')


test_df['Fongbe'] = ''

for cols in ['Fongbe','French']:
  train_df[cols] = train_df[cols].astype(str)
  valid_df[cols] = valid_df[cols].astype(str)
  test_df[cols]  = test_df[cols].astype(str)
print(train_df.shape,test_df.shape,valid_df.shape)

(55114, 4) (2929, 4) (2264, 2)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [4]:
train_df.head(3)

,ID,French,Target_Language,Fongbe
0,ID_AADNDxdl,Mon père,Fon,Tɔ ce
1,ID_AAFQhmDr,Mettez-vous en rang.,Fon,Mi tò miɖéé
2,ID_AAJfVHEH,Son doigt lui fait mal.,Fon,Alɔvi tɔn ɖo vivɛ wɛ


In [5]:
test_df.head(3)

,ID,French,Target_Language,Fongbe
1,ID_AAGuzGzi,Tous ces grands artistes viendront au Benin po...,Fon,
2,ID_AAuiTPkQ,Ce programme va travailler à améliorer les con...,Fon,
3,ID_ACYgGXTq,Quels sont les questions récurrentes de ceux ...,Fon,


## Convert the dataframes to Model readable format.

In [6]:
train_df['translation'] = train_df.apply(lambda x : {'fr':x['French'],'fon':x['Fongbe']}, axis=1)

test_df['translation']  = test_df.apply(lambda  x : {'fr':x['French'],'fon':x['Fongbe']}, axis=1)

valid_df['translation'] = valid_df.apply(lambda x : {'fr':x['French'],'fon':x['Fongbe']}, axis=1)

##### ids from test data to be used for indexing 
id2use                     = test_df['ID']


train                      = pd.DataFrame(train_df['translation'])

test                       = pd.DataFrame(valid_df['translation'])

test_dataframe             = pd.DataFrame(test_df['translation'],columns=['translation'])

#### raw dataset format
raw_datasets               = DatasetDict()
raw_datasets['train']      = Dataset.from_dict(train)
raw_datasets['validation'] = Dataset.from_dict(test)
raw_datasets['test']       = Dataset.from_dict(test_dataframe)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 55114
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2264
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2929
    })
})

## Loading the pretrained seq2seq model for the architecture.
<br>
We load the French to Ewe pretrained model.

In [8]:
metric           = load_metric("rouge")
model_checkpoint = "Helsinki-NLP/opus-mt-fr-ee"
tokenizer        = AutoTokenizer.from_pretrained(model_checkpoint)


## Dataloader creation

In [9]:
max_input_length = 256
max_target_length = 256
source_lang = "fr"
target_lang = "fon"
prefix      = ""
def preprocess_function(examples):
    inputs        = [prefix + ex[source_lang] for ex in examples["translation"]]

    targets       = [ex[target_lang] for ex in examples["translation"]]

    model_inputs  = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs


def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = metric.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])
    rouge1 = rouge_output["rouge1"].mid
    

    return {
        "rouge1_precision": round(rouge1.precision, 4),
        "rouge1_recall": round(rouge1.recall, 4),
        "rouge1_fmeasure": round(rouge1.fmeasure, 4),
    }


tokenized_datasets = raw_datasets.map(preprocess_function, batched=True,num_proc=4)

## Model training 

In [10]:
model              = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [11]:
data_collator     = DataCollatorForSeq2Seq(tokenizer, model=model)

In [12]:
args = Seq2SeqTrainingArguments("test-translation", 
                                  evaluation_strategy           = "epoch", 
                                  learning_rate                 = 4.3401933791213136e-05,
                                  seed                          = 13,
                                  per_device_train_batch_size   = 32, 
                                  per_device_eval_batch_size    = 32, 
                                  weight_decay                  = 0.0043268807715410255,
                                  save_total_limit              = 3, 
                                  num_train_epochs              = 10,
                                  metric_for_best_model         = 'rouge1_fmeasure', 
                                  dataloader_num_workers        = 16, 
                                  predict_with_generate         = True, 
                                  fp16                          = False, 
                                  greater_is_better             = True,
                                  group_by_length               = True,
                                  )

trainer = Seq2SeqTrainer(
                            model,
                            args,
                            train_dataset=tokenized_datasets["train"],
                            eval_dataset=tokenized_datasets["validation"],
                            data_collator=data_collator,
                            tokenizer=tokenizer,
                            compute_metrics=compute_metrics
                        )
trainer.train()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch,Training Loss,Validation Loss,Rouge1 Precision,Rouge1 Recall,Rouge1 Fmeasure
1,2.602900,3.129415,0.369000,0.161500,0.198300
2,2.055500,2.840167,0.329900,0.205100,0.234400
3,1.684900,2.666969,0.364600,0.251000,0.282400
4,1.488100,2.551465,0.368100,0.294800,0.310400
5,1.309700,2.502357,0.392900,0.305800,0.327900
6,1.202700,2.463231,0.398600,0.318100,0.337600
7,1.107600,2.428097,0.415600,0.324000,0.349900
8,1.045200,2.416281,0.415500,0.335600,0.356400
9,0.983300,2.404639,0.417600,0.337700,0.358600
10,0.953500,2.401324,0.416900,0.335000,0.356800


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning

TrainOutput(global_step=17230, training_loss=1.510336792558968, metrics={'train_runtime': 10512.4692, 'train_samples_per_second': 1.639, 'total_flos': 149692712140800.0, 'epoch': 10.0, 'init_mem_cpu_alloc_delta': 1719414784, 'init_mem_gpu_alloc_delta': 302624256, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -25726976, 'train_mem_gpu_alloc_delta': 902406656, 'train_mem_cpu_peaked_delta': 56700928, 'train_mem_gpu_peaked_delta': 10515992576})

## Attaining Outputs

In [13]:
trainer.compute_metrics = None
test_dataloader         = trainer.get_test_dataloader(tokenized_datasets['test'])
b                       = trainer.prediction_loop(test_dataloader,description='Prediction')
labels                  = b.label_ids
preds                   = b.predictions
decoded_preds           = tokenizer.batch_decode(preds, skip_special_tokens=True)

/usr/local/lib/python3.7/dist-packages/transformers/trainer_pt_utils.py:368: FutureWarning: DistributedTensorGatherer is deprecated and will be removed in v5 of Transformers.
  FutureWarning,


In [14]:
sub          = pd.DataFrame(id2use,columns=['ID'])
sub['Target'] = decoded_preds
sub.head()

,ID,Target
1,ID_AAGuzGzi,Jikpamátɔ ɖaxo enɛ ɔ lɛ byɔ́ wá Beɛn bó ná xwl...
2,ID_AAuiTPkQ,Nùɖéwá élɔ́ ná zɔ́n bɔ̀ gbɛtɔ́ lɛ́ jɛ̀ gbɛtɔ́ ...
3,ID_ACYgGXTq,Nù tɛ́ lɛ́ wɛ́ lɛ́ wɛ́ lɛ́ lɛ́ wɛ́ nɔ́ kàn nuk...
5,ID_AFBqjFUm,"Azɔnwe tnwe ɔ, un mɔ xwe sin ji sin xwe gba do..."
6,ID_AFFhTuyI,"Sin hwenu enɛ ɔ, nuwlannu tɔn ɖò wema gbɛtɔ́ t..."


In [15]:
sub.to_csv("french_fongbe_trial_seq2seq_salim.csv",index=False)

# Save the file attained above separately as explained in the documentation. It will be used for blending.